In [1]:
# References:
# [1] https://www.youtube.com/watch?v=Mut_u40Sqz4
# [2] https://github.com/ARISE-Initiative/robosuite/issues/131

# Dependencies

In [1]:
# !pip install stable-baselines3[extra]
# Robosuite
# gym

!echo $LD_PRELOAD
!echo $LD_LIBRARY_PATH


/usr/lib/x86_64-linux-gnu/libGLEW.so
/opt/ros/melodic/lib:/home/damnghost/.mujoco/mujoco210/bin:/home/damnghost/.mujoco/mujoco200/bin


# RL

In [2]:
import os
# openai 
import gym
# Stable baseline imports
from stable_baselines3 import PPO
from stable_baselines3.common.save_util import save_to_zip_file, load_from_zip_file
from stable_baselines3.common.monitor import Monitor
from stable_baselines3.common.vec_env import DummyVecEnv, VecNormalize
from stable_baselines3.common.evaluation import evaluate_policy
# Robosuite imports
import robosuite as suite
from robosuite.wrappers import GymWrapper
from robosuite.environments.base import register_env
from robosuite import load_controller_config

# Random Action: No Training

In [3]:
# Notice how the environment is wrapped by the wrapper
env = GymWrapper(suite.make(
                    "TwoArmPegInHole",
                    robots=["Panda", "Panda"],  # use Sawyer robot
                    use_camera_obs=False,  # do not use pixel observations
                    has_offscreen_renderer=False,  # not needed since not using pixel obs
                    has_renderer=True,  # make sure we can render to the screen
                    reward_shaping=True,  # use dense rewards
                    control_freq=20,  # control should happen fast enough so that simulation looks smooth
                            )
                )
episodes = 1
for i_episode in range(episodes):
    observation = env.reset()
    done = False
    score = 0
    while not done:
        # env.render()
        action = env.action_space.sample()
        observation, reward, done, info = env.step(action)
        score += reward
    print(f"Episode: {i_episode} Score: {score}")

env.close()

Creating window glfw
Creating window glfw


/home/damnghost/miniconda3/envs/rl_ultrasound/lib/python3.8/site-packages/gym/spaces/box.py:73: UserWarning: WARN: Box bound precision lowered by casting to float32
  logger.warn(


Creating window glfw
Cumulative reward: 343.3346532434673
Episode: 0 Score: 343.3346532434673


# Environment Specification

In [ ]:
print(f"Initial State observations: \n {env.reset()}")
print(f"Action space: \n{env.action_space}")
print(f"Observation space: \n{env.observation_space}")
env.close()

# Train RL Model

In [ ]:
# Make directories
log_dir = os.path.join('../Training', 'Logs')
print(log_dir)

In [ ]:
def wrap_env(env):
    wrapped_env = Monitor(env)                          # Needed for extracting eprewmean and eplenmean
    wrapped_env = DummyVecEnv([lambda : wrapped_env])   # Needed for all environments (e.g. used for mulit-processing)
    wrapped_env = VecNormalize(wrapped_env)             # Needed for improving training when using MuJoCo envs?
    return wrapped_env

In [ ]:
env = wrap_env(env)
model = PPO('MlpPolicy', env, verbose=1, tensorboard_log=log_dir, )

In [ ]:
model.learn(total_timesteps=1, tb_log_name="TwoArmPegInHole_PPO_TEST")

In [ ]:
PPO_path = os.path.join('../Training', 'Saved_Models', 'TwoArmPegInHole_PPO_model')
PPO_path

In [ ]:
model.save(PPO_path)
del model

In [ ]:
ENV_path = os.path.join('../Training', 'Saved_Env/')
env.save("TwoArmPegInHole_VecNorm_Env.pkl")
ENV_path